In [1]:
%matplotlib inline
import torch
import torchvision
from torch.utils import data
from torchvision import transforms
from d2l import torch as d2l
import random


d2l.use_svg_display() # 用svg来显示我们的图片, 更清晰.
# 就是再里面进行设置成svg模式.

In [4]:
num_inputs = 784 # 模型输入维度
num_outputs = 10
# 这里我们简单将每张图片看作是一维向量, 每一个像素都是x, 然后我们生成对应长度的w. 


W = torch.normal(0, 0.1, size=(num_inputs, num_outputs), requires_grad=True)

b = torch.zeros(num_outputs, requires_grad=True)

In [ ]:
# 定义softmax操作
# 首先回顾一下对于矩阵的操作